# Global indicator project - Phoenix, Arizona

## local neighborhood network accessibility analysis - amenities pois
This notebook uses sausage buffer intersection approach to calculate the accessibility score of sample point local neighborhood. This process use hard threshold distance to count whether any pois (in this case, it is shops with supermarket and convenience stores) located within the local walkable neighborhoods.

### Process:
1. Download or load points of interest (POIs) from OSM
2. Get sample point local neighborhood (sausage buffer)
3. Local neighbothood with at least one amenity pois intersecting
    - the output results are geoseries of local neighborhood buffer polygon
4. Connect the result to sample point dataframe
    - accessibility score is calculated as 1=at least one amenity pois is within the local walkable neighborhood; 0=none of the pois is within the local walkable neighborhood.




In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import numpy as np
import requests
import pandas as pd
import geopandas as gpd
import os
import time 
from shapely.geometry import shape,Point, LineString, Polygon


import pandana
from pandana.loaders import osm

ox.config(use_cache=True, log_console=True)
pandana.__path__ #pandana runs on python 3.6 kernel

['/Users/NGAU/miniconda3/envs/ind_global/lib/python3.6/site-packages/pandana']

## Set up configuration 

In [3]:
place = 'phoenix' 

region = 'Arizona, USA' # study region name

studyregion = 'Phoenix, Arizona, USA'

suffix = '_201905'

# configure search at a max distance of 1 km for up to the 10 nearest points-of-interest
shop = ['supermarket', 'convenience']

# configure filenames to save/load POI and network datasets
OSM_folder = '../data/OSM'

G_filename = '{studyregion}_walk{suffix}.graphml'.format(studyregion = studyregion, suffix = suffix)
G_proj_filename='{studyregion}_proj_walk{suffix}.graphml'.format(studyregion = place, suffix = suffix)
poi_filename = '{}_pois_{}.csv'.format(place, '_'.join(shop))

G_filepath = OSM_folder + "/" + G_filename
poi_filepath = OSM_folder + "/" + poi_filename
sample_points_filepath = '../data/OSM/phoenix_sample_points_stats_201905/phoenix_sample_points_stats_201905.shp'

In [4]:
# get bounding box from study region boundary shapefile
shape_filepath = '../data/OSM/Phoenix, Arizona, USA_buffered_201905/Phoenix, Arizona, USA_buffered_201905.shp'

gdf_shape = gpd.GeoDataFrame.from_file(shape_filepath)
bbox = [gdf_shape['bbox_south'].astype(float)[0], gdf_shape['bbox_west'].astype(float)[0], gdf_shape['bbox_north'].astype(float)[0], gdf_shape['bbox_east'].astype(float)[0]] #lat-long bounding box for Phx
bbox

[33.2903739, -112.3240289, 33.9183794, -111.9255201]

## Download points of interest (POIs) from OSM
What amenities are considered for daily living pois? - [OSMtag](https://taginfo.openstreetmap.org/keys/amenity): shop=supermarket, convenience

In [5]:
def get_osm_pois_gdf(poi_filepath=poi_filepath, shop=shop, bbox=bbox):
    if os.path.isfile(poi_filepath):
        # if a points-of-interest file already exists, just load the dataset from that
        pois = pd.read_csv(poi_filepath)
        method = 'loaded from CSV'
    else:   
        # otherwise, query the OSM API for the specified amenities within the bounding box 
        osm_tags = '"shop"~"{}"'.format('|'.join(shop))
        pois = osm.node_query(bbox[0], bbox[1], bbox[2], bbox[3], tags=osm_tags)

        # drop any that aren't just 'shop' then save to CSV
        pois = pois[pois['shop'].isin(shop)]
        pois.to_csv(poi_filepath, index=False, encoding='utf-8')
        method = 'downloaded from OSM'
    pois_df = pois[['shop', 'name', 'lat', 'lon']]
    pois_df['geometry'] = pois_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    pois_gdf = gpd.GeoDataFrame(pois_df)
    return pois_gdf
    

In [6]:
pois_gdf = get_osm_pois_gdf(poi_filepath=poi_filepath, shop=shop, bbox=bbox)
pois_gdf.head()

/Users/NGAU/miniconda3/envs/ind_global/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,shop,name,lat,lon,geometry
0,supermarket,Safeway,33.488551,-112.081794,POINT (-112.0817935 33.4885505)
1,convenience,7-Eleven,33.480013,-112.029747,POINT (-112.0297471 33.4800128)
2,supermarket,Fry's Marketplace,33.321899,-111.930303,POINT (-111.9303033 33.3218992)
3,supermarket,Fry's Food & Drug,33.379098,-111.976857,POINT (-111.9768574 33.3790977)
4,convenience,NaN,33.348719,-111.953919,POINT (-111.9539185 33.3487189)


In [44]:
#define pois points projection
pois_gdf.crs = {'init' :'epsg:4326'}
pois_gdf_proj = ox.project_gdf(pois_gdf)

## Get sample point local neighborhood (sausage buffer)

In [7]:
# load sample point dataframe
sample_points = gpd.GeoDataFrame.from_file(sample_points_filepath)
sample_points.shape

(100, 27)

In [8]:
# load study region projected graph
G_proj = ox.load_graphml(G_proj_filename, folder=OSM_folder)

In [45]:
# create list of sample points to iterate over
point_locations = []

for point in sample_points.geometry: 
    point = (point.x, point.y)
    point_locations = point_locations + [point]

In [46]:
# create sausage buffer local neighborhood
def create_sausage_buffer(G_proj, orig_point, buffer=50, length = 1600, intersection_tolerance = 15):
    # locate closest node on network to 
    orig_node = ox.get_nearest_node(G_proj, orig_point, return_dist=True)
    subgraph_proj = nx.ego_graph(G_proj, orig_node[0], radius=length, distance='length')
    # create buffer
    subgraph_gdf = ox.graph_to_gdfs(subgraph_proj, nodes=False, edges=True, fill_edge_geometry=True)
    buffer = subgraph_gdf.geometry.buffer(buffer) # create buffer 
    buffer_uu = buffer.geometry.unary_union
    return([orig_node, buffer]) #output is list of nearest origin nodes to sample points, and buffer polygon geoseries

In [47]:
start = time.time()
task = 'Buffer network for {} sample points'.format(len(point_locations))
sausagebuffers = []
for point in point_locations:
    sausagebuffers.append(create_sausage_buffer(G_proj, point))
print('Completed task "{}" in {:,.2f} seconds'.format(task,time.time() - start)) 

Completed task "Buffer network for 100 sample points" in 251.17 seconds


In [48]:
# create a dictionary and dataframe with node id as the key and buffer geoseries as values
dic_buffer_nodeid = {}
for i in range(0, len(sausagebuffers)):
    dic_buffer_nodeid[sausagebuffers[i][0][0]] = sausagebuffers[i][1]
    #buffer_nodeid_df = pd.DataFrame(dic_buffer_nodeid.items(), columns=['node_id', 'buffer_series'])

In [49]:
# create an empty list
buffer_list = []
# loop through sausage buffer intersection with pois
for x in range(0, len(sausagebuffers)):
    #Returns a Series of dtype('bool') with value True for each polygon geometry that intersects other.
    pois_intersect = sausagebuffers[x][1].intersects(pois_gdf_proj['geometry'].unary_union) 
    # append series in dataframe, return a dataframe of buffer polygon geoseries with pois intersecting
    buffer_list.append(sausagebuffers[x][1][pois_intersect])

In [70]:
pois_intersect = sausagebuffers[0][1].intersects(pois_gdf_proj['geometry'].unary_union)
sausagebuffers[0][1][pois_intersect]

766     POLYGON ((397466.005966094 3705914.451375541, ...
767     POLYGON ((397789.1916660971 3705911.16747281, ...
768     POLYGON ((397691.1035665049 3706011.960077627,...
772     POLYGON ((397739.0669513491 3706014.524117851,...
793     POLYGON ((397689.1946118103 3705910.530563731,...
962     POLYGON ((397789.0326896691 3705962.343335763,...
965     POLYGON ((397789.8455853814 3705849.956560066,...
966     POLYGON ((397689.080379246 3705964.419834015, ...
967     POLYGON ((397627.9350196733 3705861.189311151,...
968     POLYGON ((397748.5001011192 3705960.922289006,...
1415    POLYGON ((396661.5111668655 3704792.968526291,...
1456    POLYGON ((397798.3118310186 3705860.124228009,...
1482    POLYGON ((397628.0899950577 3705961.18919971, ...
1754    POLYGON ((396790.3931290578 3705114.17843892, ...
dtype: object

In [69]:
dic_buffer_nodeid.keys()

dict_keys([41797604, 1604511752, 5634577087, 2926772436, 41696680, 311167952, 1006063370, 2308625982, 795324463, 41663588, 42335982, 5773311682, 2617600069, 41884724, 5757731080, 3425015758, 3219913511, 1420663969, 356574019, 4300706048, 2915600833, 5258398824, 1924517950, 42627831, 5237362151, 41871665, 3712430301, 4600582582, 2340116750, 41496862, 4403125495, 3643509510, 3118833900, 4516480461, 4399487886, 5657807203, 1680943142, 2225727194, 6250079997, 3928557377, 42585206, 5634076538, 5656668566, 5529515849, 5646108477, 5716466625, 2328005445, 3706324835, 2030604950, 4638640560, 5683836414, 1837869136, 2892569090, 6077879130, 41570283, 1664082374, 2401950075, 5063549460, 5686534929, 41949584, 3079669227, 2294261993, 3150147376, 311169694, 41784150, 4717338673, 1509442937, 41603546, 316086812, 2293311446, 41875510, 41773025, 1895509714, 1513713005, 5280491795, 3454169480, 3911954937, 1444920885])